In [6]:
import numpy as np
import pandas as pd
import glob

# -------------------------------------------------------
# 0. Find resamplede filer
# -------------------------------------------------------
files = sorted(glob.glob("resampled_pretty/*_resampled_pretty.csv"))
print("Resamplede filer fundet:", len(files))

# -------------------------------------------------------
# 1. MCU-safe preprocess
# -------------------------------------------------------
def preprocess(df):
    for axis in ["accX", "accY", "accZ"]:
        df[axis] = df[axis].astype(np.float32).round(4)
    return df

# -------------------------------------------------------
# 2. Sliding windows
# -------------------------------------------------------
def make_windows(df, window_seconds=2, fs=100):
    window_size = window_seconds * fs      # 200 samples
    step = window_size // 2                # 100 overlap

    assert window_size == 200

    data = df.loc[:, ["accX", "accY", "accZ"]].values.astype(np.float32)
    label = df["label"].iloc[0]

    X, y = [], []
    for start in range(0, len(data) - window_size + 1, step):
        X.append(data[start:start + window_size])
        y.append(label)

    return X, y

# -------------------------------------------------------
# 3. Loop gennem alle filer
# -------------------------------------------------------
X_all, y_all = [], []

for file in files:
    print("Behandler:", file)
    df = pd.read_csv(file)

    df_p = preprocess(df)
    X, y = make_windows(df_p)

    X_all.extend(X)
    y_all.extend(y)

# -------------------------------------------------------
# 4. Endelige arrays
# -------------------------------------------------------
X_final = np.array(X_all, dtype=np.float32)
y_final = np.array(y_all)

np.save("X_windows.npy", X_final)
np.save("y_labels.npy", y_final)

print("\n✔ MCU-safe preprocess færdig!")
print("X_windows.npy shape:", X_final.shape)
print("y_labels.npy shape:", y_final.shape)
print("Labels:", np.unique(y_final))


Resamplede filer fundet: 27
Behandler: resampled_pretty\Normal_kørsel0002_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0003_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0004_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0005_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0006_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0007_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0013_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0014_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0015_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0016_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0017_resampled_pretty.csv
Behandler: resampled_pretty\Normal_kørsel0018_resampled_pretty.csv
Behandler: resampled_pretty\Udryknings_kørsel0001_resampled_pretty.csv
Behandler: resampled_pretty\Udryknings_kørsel0002_resampled_pretty.csv
Behandler: resampled_prett

In [4]:
X = np.load("X_windows.npy")
y = np.load("y_labels.npy")

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Unique labels:", np.unique(y))

X shape: (757, 200, 3)
y shape: (757,)
Unique labels: ['normal' 'udrykning']
